## Sample code for generating pair matrix from patent data
- https://www.kaggle.com/code/rahulgoel1106/network-analysis-using-igraph

In [4]:
# !pip install python-igraph
# pip install matplotlib

In [ ]:
# !pip uninstall cairocffi  # convince pip to actually do the next step despite identical version numbers
# !pip install git+https://github.com/SimonSapin/cairocffi.gita

##### Import library
- If you are Mac user and run this code in Jupyter notebook, install brew and runt the following command line. (Ref: https://www.csestack.org/macbook-m1-oserror-no-library-called-cairo-pango-was-found/)

- brew install cairo pango gdk-pixbuf libxml2 libxslt libffi
- brew install cairo
- brew install py2cairo
- brew install igraph  (install C-core)
- sudo pip install python-igraph (install igraph for python)

##### Error importing cairo
#### brew
- Open terminal and reinstall (ref:https://github.com/Automattic/node-canvas/wiki/Installation%3A-Mac-OS-X)
- curl -L http://www.cairographics.org/releases/cairo-1.12.18.tar.xz -o cairo.tar.xz
- tar -xf cairo.tar.xz && cd cairo-1.12.18
- ./configure --prefix=/usr/local --disable-dependency-tracking
- make install

#### conda environment
- conda install cairo pango gdk-pixbuf libffi cairosvg
- cairosvg image.svg -o image.png

In [5]:
import pandas as pd
from itertools import combinations

In [ ]:
# conda install cairo

In [3]:
import igraph as ig

##### Import data set

In [6]:
data = pd.read_csv("../data/network_python/network_sample.csv")
data.head()

,Unnamed: 0,permno_adj,name_std,year,3_year,appln_id,kind,person_id,applt_seq_nr,invt_seq_nr,person_name,doc_std_name,psn_name,psn_level,psn_sector
0,0,76129,3COM CORP,1984,2,47299655,D,37298464.0,0.0,4.0,HAWKINS JEFFREY C.,HAWKINS JEFFREY C,HAWKINS JEFFREY C.,0.0,NaN
1,1,76129,3COM CORP,1984,2,47299655,D,37737272.0,0.0,3.0,ELLIOTT STEVEN O.,ELLIOTT STEVEN O,ELLIOTT STEVEN O.,0.0,NaN
2,2,76129,3COM CORP,1984,2,47299655,D,37737271.0,0.0,2.0,GROVE DALLAS,GROVE DALLAS,GROVE DALLAS,0.0,NaN
3,3,76129,3COM CORP,1984,2,47299655,D,37737276.0,0.0,8.0,CHEBELEU LIVIUS DUMITRU,CHEBELEU LIVIUS DUMITRU,CHEBELEU LIVIUS DUMITRU,0.0,NaN
4,4,76129,3COM CORP,1984,2,47299655,D,37737277.0,0.0,9.0,SANDHU KULBIR S.,SANDHU KULBIR S,SANDHU KULBIR S.,0.0,NaN


##### Create pair matrix

In [7]:
data_1 = data[['appln_id','person_id','psn_name','psn_level']]
data_1.head()

,appln_id,person_id,psn_name,psn_level
0,47299655,37298464.0,HAWKINS JEFFREY C.,0.0
1,47299655,37737272.0,ELLIOTT STEVEN O.,0.0
2,47299655,37737271.0,GROVE DALLAS,0.0
3,47299655,37737276.0,CHEBELEU LIVIUS DUMITRU,0.0
4,47299655,37737277.0,SANDHU KULBIR S.,0.0


In [8]:
# node table
data_node = data_1[['person_id','psn_name','psn_level']].drop_duplicates()
data_node.reset_index(drop=True, inplace=True)
#data_node = data_node.set_index(['psn_name'])
data_node.head()

,person_id,psn_name,psn_level
0,37298464.0,HAWKINS JEFFREY C.,0.0
1,37737272.0,ELLIOTT STEVEN O.,0.0
2,37737271.0,GROVE DALLAS,0.0
3,37737276.0,CHEBELEU LIVIUS DUMITRU,0.0
4,37737277.0,SANDHU KULBIR S.,0.0


In [9]:
def get_combinations(group):
    return pd.DataFrame([sorted(e) for e in list(combinations(group['person_id'].values, 2))], columns=['from', 'to'])

data_1_gr = data_1.groupby('appln_id').apply(get_combinations)
data_1_gr

from          to
appln_id                           
467875    0   5228761.0   5228762.0
46894286  0  18671816.0  37439821.0
          1  18671816.0  37439820.0
          2  18671816.0  33395526.0
          3  37439820.0  37439821.0
...                 ...         ...
506368180 2  58138186.0  60418379.0
          3  16001514.0  16092391.0
          4  16001514.0  58138186.0
          5  16092391.0  58138186.0
506368232 0  16001514.0  16131550.0

[664 rows x 2 columns]

In [10]:
# edge table
data_pair = data_1_gr.groupby(['from', 'to']).size().to_frame('weight').reset_index()
data_pair.head(10)

,from,to,weight
0,5228761.0,5228762.0,1
1,5690226.0,5690227.0,1
2,5690226.0,5690228.0,1
3,5690226.0,5690230.0,1
4,5690226.0,6487867.0,1
5,5690226.0,6746347.0,1
6,5690226.0,6746348.0,1
7,5690227.0,5690230.0,1
8,5690227.0,6746347.0,1
9,5690227.0,6746348.0,1


In [11]:
data_node.to_csv("../data/network_python/network_sample_node.csv")
data_pair.to_csv("../data/network_python/network_sample_pair.csv")